In [1]:
# 🧹 Clean MinerU Markdown - Batch Version (Simple)

from pathlib import Path
import re

def remove_hkma_tables(text: str) -> tuple[str, int]:
    """Remove HKMA header and footer tables"""
    table_re = re.compile(r"<table\b.*?>.*?</table>", re.DOTALL | re.IGNORECASE)
    
    removed = 0
    parts = []
    last_end = 0

    for m in table_re.finditer(text):
        table_html = m.group(0)
        
        # Check if it's a header or footer table
        is_header = ("HONG KONG MONETARY AUTHORITY" in table_html and 
                    "Supervisory Policy Manual" in table_html)
        is_footer = ("Contents" in table_html and 
                    "Glossary" in table_html and 
                    "Home" in table_html)
        
        if is_header or is_footer:
            parts.append(text[last_end:m.start()])
            last_end = m.end()
            removed += 1
        
    parts.append(text[last_end:])
    cleaned = "".join(parts)
    cleaned = re.sub(r"\n{3,}", "\n\n", cleaned)  # Clean up blank lines
    
    return cleaned, removed


# === 批次處理所有檔案 ===
input_folder = Path("minerU_parse")  # 改成你的資料夾名稱
output_folder = input_folder / "cleaned"  # 輸出到 cleaned 子資料夾
output_folder.mkdir(exist_ok=True)

# 找出所有 .md 檔案（排除已清理的）
md_files = [f for f in input_folder.glob("*.md") if ".clean" not in f.stem]

print(f"🔍 Found {len(md_files)} files\n")

for i, file in enumerate(md_files, 1):
    print(f"[{i}/{len(md_files)}] {file.name}")
    
    # 讀取、清理、儲存
    text = file.read_text(encoding="utf-8")
    cleaned, removed = remove_hkma_tables(text)
    
    output_file = output_folder / f"{file.stem}.clean.md"
    output_file.write_text(cleaned, encoding="utf-8")
    
    print(f"   ✅ Removed {removed} tables → {output_file.name}\n")

print(f"✨ Done! All files saved to: {output_folder}")

🔍 Found 5 files

[1/5] AML-1_MinerU__20251019054235.md
   ✅ Removed 24 tables → AML-1_MinerU__20251019054235.clean.md

[2/5] AML-2_MinerU__20251019061151.md
   ✅ Removed 0 tables → AML-2_MinerU__20251019061151.clean.md

[3/5] CG-5_MinerU__20251017075335.md
   ✅ Removed 29 tables → CG-5_MinerU__20251017075335.clean.md

[4/5] IC-1_MinerU__20251017042622.md
   ✅ Removed 43 tables → IC-1_MinerU__20251017042622.clean.md

[5/5] SA-1_MinerU__20251017152031.md
   ✅ Removed 32 tables → SA-1_MinerU__20251017152031.clean.md

✨ Done! All files saved to: minerU_parse\cleaned
